# Fukuhara-Seminar-Week-01: Python Analysis Reviewing Textbook

## 0.0: Contents Table
- Data Colecting
  - データの概要とか、期間や種類とか、選んだ理由ちょいと
- Regression & Hypothesis Testing
  - CAPMのベータ計算：
  - 重回帰分析：
- Asuumption
  - (Zero mean)
  - Homoscedasticy
    - breusc-pagan
  - No Autocorrelation(DW)
  - Exogeneity(dollow from non-stochastic X)...N/A out of scope
    - Cov(u, X)=0
  - Normality→python guide
    - jarque bera
  - Linearity
    - RESET
  - Stability
    - cusum
- Minimum Variance Portofolio and Market Portofolio
  - Portfolio theory
- (Copula)


## 1.0: Make monthly stock dataset for 10 years

In [80]:
import pandas as pd
from pandas_datareader import data as web
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import yfinance as yf
import statsmodels.stats.api as sms
from statsmodels.compat import lzip
from scipy.optimize import minimize
import cvxpy as cp

# ターゲットとなる企業のティッカーと企業名のリスト
tickers = [
    "1925", "1928", "2413", "2502", "2503", "2802", "2914", "3382", "3402", "3407", "4063", "4188", "4452", "4502", "4503", "4507", "4519", "4523", "4528", "4543", "4568", "4578", "4661", "4689", "4901", "4911", "5020", "5108", "5401", "5713", "5802", "6098", "6178", "6273", "6301", "6326", "6367", "6501", "6502", "6503", "6586", "6594", "6645", "6702", "6752", "6758", "6861", "6869", "6902", "6920", "6954", "6971", "6981", "7011", "7201", "7203", "7267", "7269", "7270", "7309", "7733", "7741", "7751", "7832", "7974", "8001", "8002", "8031", "8035", "8053", "8058", "8113", "8267", "8306", "8308", "8309", "8316", "8411", "8591", "8604", "8630", "8697", "8725", "8750", "8766", "8801", "8802", "8830", "9020", "9021", "9022", "9101", "9202", "9432", "9433", "9434", "9735", "9843", "9983", "9984", "TOPIX100", '1475'
]

companies = [
    '大和ハウス工業','積水ハウス','エムスリー','アサヒグループホールディングス','キリンホールディングス','味の素','日本たばこ産業','セブン＆アイ・ホールディングス','東レ','旭化成','信越化学工業','三菱ケミカルホールディングス','花王','武田薬品工業','アステラス製薬','塩野義製薬','中外製薬','エーザイ','小野薬品工業','テルモ','第一三共','大塚ホールディングス','オリエンタルランド','Ｚホールディングス','富士フイルムホールディングス','資生堂','JXTGホールディングス','ブリヂストン','日本製鉄','住友金属鉱山','住友電気工業','リクルートホールディングス','日本郵政','SMC','小松製作所','クボタ','ダイキン工業','日立製作所','東芝','三菱電機','マキタ','日本電産','オムロン','富士通','パナソニック','ソニー','キーエンス','シスメックス','デンソー','レーザーテック','銘柄名','ファナック','京セラ','村田製作所','三菱重工業','日産自動車','トヨタ自動車','本田技研工業','スズキ','SUBARU','シマノ','オリンパス','HOYA','キヤノン','バンダイナムコホールディングス','任天堂','伊藤忠商事','丸紅','三井物産','東京エレクトロン','住友商事','三菱商事','ユニ・チャーム','イオン','三菱UFJフィナンシャル・グループ','りそなホールディングス','三井住友トラスト・ホールディングス','三井住友フィナンシャルグループ','みずほフィナンシャルグループ','オリックス','野村ホールディングス','SOMPOホールディングス','日本取引所グループ','MS＆ADインシュアランスグループホールディングス','第一生命ホールディングス','東京海上ホールディングス','三井不動産','三菱地所','住友不動産','東日本旅客鉄道','西日本旅客鉄道','東海旅客鉄道','日本郵船','ANAホールディングス','日本電信電話','KDDI','ソフトバンク','セコム','ニトリホールディングス','ファーストリテイリング','ソフトバンクグループ', 'TOPIX100', 'TOPIX'
]

In [ ]:
import yfinance as yf
import pandas as pd

# TOPIX100の銘柄のティッカーシンボルリストへの変換
tickers = [tickers + '.T' for tickers in tickers]

# 10年前の日付を取得
end_date = pd.Timestamp.now()
start_date = end_date - pd.DateOffset(years=10)

# 結果を格納するDataFrameの初期化
result_df = pd.DataFrame()

for ticker in tickers:
    try:
        # データの取得
        data = yf.download(ticker, start=start_date, end=end_date, interval="1mo")
        
        # データの終値を結果のDataFrameに追加
        result_df[ticker] = data["Close"]
    except Exception as e:
        print(f"Error fetching data for {ticker}. Error: {e}")

# Dateを1列目に設定
result_df.reset_index(inplace=True)

# CSVに保存
result_df.to_csv("topix100_monthly_data.csv", index=False)

print("Data fetching and saving completed!")

In [82]:
df = pd.read_csv('./topix100_monthly_data.csv')
df

,Date,1925.T,1928.T,2413.T,2502.T,2503.T,2802.T,2914.T,3382.T,3402.T,...,9202.T,9432.T,9433.T,9434.T,9735.T,9843.T,9983.T,9984.T,TOPIX100.T,1475.T
0,2013-11-01,1990.0,1413.0,668.25,2804.0,1578.0,1460.0,3460.0,3770.0,724.000000,...,2070.0,51.400002,2143.333252,NaN,6320.0,4745.0,12933.333008,4145.0,NaN,NaN
1,2013-12-01,2035.0,1470.0,658.75,2964.0,1513.0,1522.0,3420.0,4180.0,728.000000,...,2100.0,56.599998,2156.666748,NaN,6340.0,4985.0,14466.666992,4600.0,NaN,NaN
2,2014-01-01,1964.0,1434.0,756.25,2813.0,1406.0,1454.0,3197.0,4102.0,677.000000,...,2180.0,55.610001,1901.666626,NaN,5830.0,4985.0,12703.333008,3781.5,NaN,NaN
3,2014-02-01,1846.0,1274.0,846.25,2852.0,1385.0,1577.0,3232.0,3813.0,701.000000,...,2270.0,57.009998,2067.000000,NaN,5737.0,4600.0,11683.333008,3834.0,NaN,NaN
4,2014-03-01,1751.0,1281.0,847.00,2889.0,1430.0,1475.0,3240.0,3944.0,682.000000,...,2230.0,56.189999,1992.000000,NaN,5946.0,4475.0,12478.333008,3900.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,2023-06-01,3792.0,2905.5,3109.00,5574.0,2101.5,5724.0,3153.0,6205.0,800.400024,...,3426.0,170.500000,4450.000000,1540.0,9739.0,16105.0,36720.000000,6773.0,1539.900024,2389.0
116,2023-07-01,3862.0,2899.0,3263.00,5595.0,2102.5,5537.0,3153.0,5893.0,795.200012,...,3406.0,162.899994,4187.000000,1578.0,9533.0,17455.0,35560.000000,7242.0,1553.890015,2401.0
117,2023-08-01,4048.0,2970.5,2903.50,5676.0,2046.0,6173.0,3191.0,5981.0,786.099976,...,3290.0,168.300003,4325.000000,1670.0,10195.0,16620.0,33480.000000,6545.0,1550.140015,2384.0
118,2023-09-01,4015.0,2977.0,2716.00,5590.0,2093.0,5764.0,3440.0,5855.0,778.000000,...,3133.0,176.600006,4577.000000,1690.5,10140.0,16710.0,32590.000000,6335.0,1550.079956,2401.0


In [ ]:
# 各銘柄のデータを数値に変換
for column in df.columns:
    if column != 'Date':  # Date列はスキップ
        df[column] = pd.to_numeric(df[column], errors='coerce')

# 日次収益率を計算する関数
def compute_daily_return(price_column):
    return price_column.pct_change().fillna(0)

# 各銘柄の日次収益率を計算
return_columns = []
for column in df.columns:
    if column != 'Date':  # Date列はスキップ
        return_col_name = f'Return_{column}'
        df[return_col_name] = compute_daily_return(df[column])
        return_columns.append(return_col_name)

# 100銘柄全体の平均収益率を計算
df['TOPIX100_return'] = df[return_columns].mean(axis=1)

In [84]:
df['7203.T']

0      1276.000000
1      1284.000000
2      1184.400024
3      1167.800049
4      1165.199951
          ...     
115    2308.500000
116    2386.000000
117    2515.000000
118    2677.500000
119    2573.000000
Name: 7203.T, Length: 120, dtype: float64

In [85]:
from sklearn.linear_model import LinearRegression

# 月次リターンを計算
df['Toyota_Return'] = df['7203.T'].pct_change()
df['Market_Return'] = df['TOPIX100.T'].pct_change()

# NaNを除去
df = df.dropna()

# リターンを線形回帰の入力として整形
X = df['Market_Return'].values.reshape(-1, 1)
y = df['Toyota_Return'].values

# 線形回帰を実施
reg = LinearRegression().fit(X, y)

# ベータを取得
beta = reg.coef_[0]

print(f"Toyota's Market Beta: {beta}")

Toyota's Market Beta: 1.0514510964657375


In [ ]:
import statsmodels.api as sm

# リスクフリーレート(日本国債10年の利回り)
risk_free_data = 0.008

# 日次収益率の計算
df['TOPIX100_return'] = df['TOPIX100.T'].pct_change().dropna()
df['TOYOTA_return'] = df['7203.T'].pct_change().dropna()

# 日付でデータをマージ
df_all = pd.merge(df, risk_free_data, on='date')

# 超過収益率の計算
df_all['excess_return_toyota'] = df_all['トヨタ自動車_return'] - df_all['risk_free_rate']
df_all['excess_return_topix'] = df_all['TOPIX100_return'] - df_all['risk_free_rate']

# 回帰分析
X = df_all['excess_return_topix']
X = sm.add_constant(X)
y = df_all['excess_return_toyota']

model = sm.OLS(y, X).fit()
beta = model.params['excess_return_topix']

print("Toyota's Market Beta:", beta)

In [87]:
import statsmodels.api as sm
Rf = 0.008 #日本10年国債

TOYOTA = LinearRegression()
X = df["1475.T"] - Rf
X = sm.add_constant(X)
y = df["7203.T"] - Rf #print(X)
#print(y)
TOYOTA = sm.OLS(y, X).fit()
TOYOTA.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 7203.T   R-squared:                       0.757
Model:                            OLS   Adj. R-squared:                  0.751
Method:                 Least Squares   F-statistic:                     127.9
Date:                Fri, 06 Oct 2023   Prob (F-statistic):           3.53e-14
Time:                        12:40:36   Log-Likelihood:                -283.25
No. Observations:                  43   AIC:                             570.5
Df Residuals:                      41   BIC:                             574.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -876.7588    246.657     -3.555      0.001   -1374.892    -378.626
1475.T         1.4093      0.125     11.307      0.000       1.158       1.661
==============================================================================
Omnibus:                        1.429   Durbin-Watson:                   0.283
Prob(Omnibus):                  0.490   Jarque-Bera (JB):                1.294
Skew:                           0.279   Prob(JB):                        0.524
Kurtosis:                       2.359   Cond. No.                     1.78e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.78e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [89]:
import statsmodels.api as sm
Rf = 0.008 #日本10年国債

NINTENDO = LinearRegression()
X = df["1475.T"] - Rf
X = sm.add_constant(X)
y = df["7974.T"] - Rf #print(X)
#print(y)
NINTENDO = sm.OLS(y, X).fit()
NINTENDO.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 7974.T   R-squared:                       0.271
Model:                            OLS   Adj. R-squared:                  0.253
Method:                 Least Squares   F-statistic:                     15.23
Date:                Fri, 06 Oct 2023   Prob (F-statistic):           0.000347
Time:                        12:42:10   Log-Likelihood:                -327.01
No. Observations:                  43   AIC:                             658.0
Df Residuals:                      41   BIC:                             661.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       3111.3430    682.514      4.559      0.000    1732.977    4489.709
1475.T         1.3459      0.345      3.903      0.000       0.649       2.042
==============================================================================
Omnibus:                        0.697   Durbin-Watson:                   0.483
Prob(Omnibus):                  0.706   Jarque-Bera (JB):                0.717
Skew:                           0.008   Prob(JB):                        0.699
Kurtosis:                       2.367   Cond. No.                     1.78e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.78e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [94]:
import statsmodels.api as sm
Rf = 0.008 #日本10年国債

SISEIDO = LinearRegression()
X = df["1475.T"] - Rf
X = sm.add_constant(X)
y = df["4911.T"] - Rf #print(X)
#print(y)
SISEIDO = sm.OLS(y, X).fit()
SISEIDO.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 4911.T   R-squared:                       0.017
Model:                            OLS   Adj. R-squared:                 -0.007
Method:                 Least Squares   F-statistic:                    0.7052
Date:                Fri, 06 Oct 2023   Prob (F-statistic):              0.406
Time:                        12:47:15   Log-Likelihood:                -349.31
No. Observations:                  43   AIC:                             702.6
Df Residuals:                      41   BIC:                             706.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       7415.9528   1146.482      6.468      0.000    5100.586    9731.320
1475.T        -0.4865      0.579     -0.840      0.406      -1.656       0.683
==============================================================================
Omnibus:                        1.898   Durbin-Watson:                   0.374
Prob(Omnibus):                  0.387   Jarque-Bera (JB):                1.525
Skew:                           0.290   Prob(JB):                        0.467
Kurtosis:                       2.282   Cond. No.                     1.78e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.78e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [95]:
import scipy.stats as stats
for model in [TOYOTA, NINTENDO, SISEIDO]:
    coefficient_to_test = model.params[1]
    
    # Define the null hypothesis (H0) and alternative hypothesis (H1
    null_hypothesis = 1 # Coefficient = 1 under the null hypothesis # Compute the standard error of the coefficient estimate
    std_err = model.bse[1]
    
    # Compute the t-statistic
    
    t_statistic = (coefficient_to_test - null_hypothesis) / std_err # Calculate the degrees of freedom
    df = model.df_resid
    
    # Calculate the p-value for a one-sided t-test
    p_value = 1 - stats.t.cdf(np.abs(t_statistic), df)
    
    # Check if the p-value is less than your significance level (e.g
    alpha_levels = [0.01, 0.05, 0.1]
    if model == TOYOTA:
        print("TOYOTA")
    elif model == NINTENDO:
        print("NINTENDO")
    else:
        print("SISEIDO")
    
    # Print the coefficient and test results for different significance levels
    for alpha in alpha_levels:
        if p_value < alpha:
            print(f"Significance Level: {alpha:.2f} - Reject the null hypothesis. Coefficient ({coefficient_to_test:.4f}) is greater than 1.")
        else:
            print(f"Significance Level: {alpha:.2f} - Fail to reject the null hypothesis. Coefficient ({coefficient_to_test:.4f}) is not greater than 1.")
    # Display the t-statistic and p-value

    print(f"Coefficient: {coefficient_to_test:.4f}")
    print(f"t-statistic: {t_statistic:.4f}")
    print(f"p-value: {p_value:.4f}")
    print("\n")
    

TOYOTA
Significance Level: 0.01 - Reject the null hypothesis. Coefficient (1.4093) is greater than 1.
Significance Level: 0.05 - Reject the null hypothesis. Coefficient (1.4093) is greater than 1.
Significance Level: 0.10 - Reject the null hypothesis. Coefficient (1.4093) is greater than 1.
Coefficient: 1.4093
t-statistic: 3.2838
p-value: 0.0011


NINTENDO
Significance Level: 0.01 - Reject the null hypothesis. Coefficient (-0.4865) is greater than 1.
Significance Level: 0.05 - Reject the null hypothesis. Coefficient (-0.4865) is greater than 1.
Significance Level: 0.10 - Reject the null hypothesis. Coefficient (-0.4865) is greater than 1.
Coefficient: -0.4865
t-statistic: -2.5660
p-value: 0.0070


SISEIDO
Significance Level: 0.01 - Reject the null hypothesis. Coefficient (-0.4865) is greater than 1.
Significance Level: 0.05 - Reject the null hypothesis. Coefficient (-0.4865) is greater than 1.
Significance Level: 0.10 - Reject the null hypothesis. Coefficient (-0.4865) is greater than 